In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from copy import deepcopy
from ct_assignment import (marker_localization, reverse_dict)
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})


## 1 Load and transform data

In [ ]:
df = pd.read_csv(os.path.join("data", "gt_pred_ie_consolidated.csv"), index_col=0)
df_subset = df[df["gt_noisy"] != 2]
df_subset = df_subset[df_subset["gt_proofread"] < 2]
df_subset["marker localization"] = df_subset["channel"].apply(lambda x: reverse_dict(marker_localization)[x])

In [ ]:
# make visualization of cell wise f1 score
thresh = 0.5
df_subset["tp"] = np.logical_and(df_subset["gt_proofread"] == 1, df_subset["nimbus"] > thresh).astype(int)
df_subset["fp"] = np.logical_and(df_subset["gt_proofread"] == 0, df_subset["nimbus"] > thresh).astype(int)
df_subset["fn"] = np.logical_and(df_subset["gt_proofread"] == 1, df_subset["nimbus"] <= thresh).astype(int)
df_subset["tn"] = np.logical_and(df_subset["gt_proofread"] == 0, df_subset["nimbus"] <= thresh).astype(int)
precision_nimbus = df_subset.groupby("dataset").sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fp"]), axis=1).sort_values(ascending=False)
recall_nimbus = df_subset.groupby("dataset").sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fn"]), axis=1).sort_values(ascending=False)
specificity_nimbus = df_subset.groupby("dataset").sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tn"]/(x["tn"] + x["fp"]), axis=1).sort_values(ascending=False)
f1_nimbus = 2 * precision_nimbus * recall_nimbus / (precision_nimbus + recall_nimbus)
nimbus_metric_df = pd.DataFrame({"Precision": precision_nimbus, "Recall": recall_nimbus, "Specificity": specificity_nimbus, "F1 score": f1_nimbus}).reset_index()

In [ ]:
precision = df_subset.groupby(["dataset", "plot_lineage"]).sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fp"]), axis=1).sort_values(ascending=False)
recall = df_subset.groupby(["dataset", "plot_lineage"]).sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fn"]), axis=1).sort_values(ascending=False)
specificity = df_subset.groupby(["dataset", "plot_lineage"]).sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tn"]/(x["tn"] + x["fp"]), axis=1).sort_values(ascending=False)
f1 = 2 * precision * recall / (precision + recall)
nimbus_lineage = pd.DataFrame({"Precision": precision, "Recall": recall, "Specificity": specificity, "F1 score": f1})

In [ ]:
# make visualization of cell wise f1 score
noisy_df = deepcopy(df_subset)
noisy_df["tp"] = np.logical_and(noisy_df["gt_proofread"] == 1, noisy_df["gt_noisy"] == 1).astype(int)
noisy_df["fp"] = np.logical_and(noisy_df["gt_proofread"] == 0, noisy_df["gt_noisy"] == 1).astype(int)
noisy_df["fn"] = np.logical_and(noisy_df["gt_proofread"] == 1, noisy_df["gt_noisy"] == 0).astype(int)
noisy_df["tn"] = np.logical_and(noisy_df["gt_proofread"] == 0, noisy_df["gt_noisy"] == 0).astype(int)
precision_ss = noisy_df.groupby("dataset").sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fp"]), axis=1).sort_values(ascending=False)
recall_ss = noisy_df.groupby("dataset").sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fn"]), axis=1).sort_values(ascending=False)
specificity_ss = noisy_df.groupby("dataset").sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tn"]/(x["tn"] + x["fp"]), axis=1).sort_values(ascending=False)
f1_ss = 2 * precision_ss * recall_ss / (precision_ss + recall_ss)
noisy_metric_df = pd.DataFrame({"Precision": precision_ss, "Recall": recall_ss, "Specificity": specificity_ss, "F1 score": f1_ss}).reset_index()

In [ ]:
precision = noisy_df.groupby(["dataset", "plot_lineage"]).sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fp"]), axis=1).sort_values(ascending=False)
recall = noisy_df.groupby(["dataset", "plot_lineage"]).sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tp"]/(x["tp"] + x["fn"]), axis=1).sort_values(ascending=False)
specificity = noisy_df.groupby(["dataset", "plot_lineage"]).sum(["tp", "fp", "fn", "tn"]).apply(lambda x: x["tn"]/(x["tn"] + x["fp"]), axis=1).sort_values(ascending=False)
f1 = 2 * precision * recall / (precision + recall)
noisy_lineage = pd.DataFrame({"Precision": precision, "Recall": recall, "Specificity": specificity, "F1 score": f1})

#### Figure 4 a

In [ ]:
out_dir = "figures/figure_4"
os.makedirs(out_dir, exist_ok=True)
fig_name = "f1_score_gold-vs-silver.svg" 


nimbus_metric_df["Model"] = "Nimbus"
noisy_metric_df["Model"] = "Silver"
metric_df = pd.concat([nimbus_metric_df, noisy_metric_df])
metric_df = metric_df.melt(id_vars=["dataset", "Model"], value_vars=["Precision", "Recall", "Specificity", "F1 score"], var_name="Metric", value_name="Score")
metric_df.sort_values(by="Score", ascending=False, inplace=True)
# add jitter plot in the background
sns.stripplot(data=metric_df, y="Score", x="Metric", hue="Model", palette=["Lightblue", "Lightgray"], alpha=0.85,
    order=["Specificity", "Precision", "Recall", "F1 score"]
)

ax = sns.barplot(data=metric_df, y="Score", x="Metric", hue="Model" ,palette=["Darkblue", "Gray"],
                ci=None, order=["Specificity", "Precision", "Recall", "F1 score"],
)
ax.set_ylim(0.0,1)
ax.set_title("Performance of Nimbus vs. Gold-Standard")
ax.set(xlabel='Metric', ylabel='Value')
# add legend to top right
ax.legend(["Nimbus", "Silver Standard"], loc="upper right")
# make plot larger
plt.gcf().set_size_inches(5, 5)
plt.savefig(os.path.join(out_dir, fig_name), format='svg')
plt.show()

#### Figure 4 b

In [ ]:
fig_name = "f1_score_split_by_tissue.svg" 

df_tmp = pd.concat([nimbus_metric_df, noisy_metric_df])
df_tmp = df_tmp.melt(
    id_vars=["dataset", "Model", "tissue"], value_vars=["Precision", "Recall", "Specificity", "F1 score"], var_name="Metric", value_name="Score"
)
df_tmp.tissue.replace({"pancreas": "Pancreas", "breast": "Breast", "decidua": "Decidua", "colon": "Colon"}, inplace=True)
df_tmp = df_tmp[df_tmp["Metric"] == "F1 score"]
fig, ax = plt.subplots(1,1,figsize=(4,4))
# add jitter plot in the background
sns.stripplot(data=df_tmp, y="Score", x="tissue", hue="Model", palette=["Lightblue", "Lightgray"], alpha=0.85, ax=ax, legend=False,
    order=["Pancreas", "Breast", "Decidua", "Colon"]
)
sns.barplot(
    data=df_tmp, y="Score", x="tissue", hue="Model", palette=["Darkblue", "Gray"], ci=None, ax=ax,
    order=["Pancreas", "Breast", "Decidua", "Colon"]
)
ax.set_title("F1 Score split by Tissue Type")
ax.set_ylim(0.0,1)
ax.set(xlabel='Tissue Type', ylabel='F1 Score')
plt.tight_layout()
# add legend to top right
ax.legend(["Nimbus", "Silver Standard"], loc="upper right")
plt.gcf().set_size_inches(5, 5)

plt.savefig(os.path.join(out_dir, fig_name), format='svg')
plt.show()



#### Figure 4 c

In [ ]:
fig_name = "f1_score_split_by_lineage.svg" 

nimbus_lineage_tmp = nimbus_lineage.reset_index()
noisy_lineage_tmp = noisy_lineage.reset_index()
nimbus_lineage_tmp["Model"] = "Nimbus"
noisy_lineage_tmp["Model"] = "Silver"
df_tmp = pd.concat([nimbus_lineage_tmp, noisy_lineage_tmp])
df_tmp = df_tmp[df_tmp["plot_lineage"] != "Pan-Negative"].reset_index()

order = ["Cancer", "Epithelial", "Lymphocytes", "Myeloids", "Stroma", "Muscle", "Other Immune", "Vasculature", "Precursors", "Other"]

sns.stripplot(
    data=df_tmp, y="F1 score", x="plot_lineage", hue="Model", palette=["Lightblue", "Lightgray"], alpha=0.85, order=order,
)
ax = sns.barplot(
    data=df_tmp, y="F1 score", x="plot_lineage", hue="Model", palette=["Darkblue", "Gray"], ci=None, order=order,
)
plt.xticks(rotation=90)
ax.set_title("F1 Score split by Cell Lineage")
ax.set_ylim(0.0,1)
ax.set(xlabel='Lineage', ylabel='F1 Score')
plt.tight_layout()
# add legend to top right
ax.legend(["Nimbus", "Silver Standard"], loc="upper right")
plt.gcf().set_size_inches(5, 5)

plt.savefig(os.path.join(out_dir, fig_name), format='svg')
plt.show()

#### Supplement Figure 1 h

In [ ]:
noisy_metric_df_flat = noisy_metric_df[["dataset", "Precision", "Recall", "Specificity", "F1 score"]].melt(id_vars=["dataset"])
noisy_metric_df_flat["model"] = "Silver"
nimbus_metric_df_flat = nimbus_metric_df[["dataset", "Precision", "Recall", "Specificity", "F1 score"]].melt(id_vars=["dataset"])
nimbus_metric_df_flat["model"] = "Nimbus"

In [ ]:
out_dir = "figures/supplement"
os.makedirs(out_dir, exist_ok=True)

plot_df = pd.concat([noisy_metric_df_flat, nimbus_metric_df_flat])
plot_df["Metric"] = plot_df.apply(lambda x: x["variable"] + "_" + x["model"], axis=1)
plot_df.rename(columns={"dataset": "Dataset"}, inplace=True)
plot_df.Dataset.replace(
    {"mibi_decidua": "MIBI-TOF Decidua", "codex_colon": "CODEX Colon", "vectra_pancreas": "Vectra\n Pancreas",
     "vectra_colon": "Vectra\n Colon", "mibi_breast": "MIBI-TOF Breast"},
    inplace=True
)
# plot grouped bars
fig_name = "f1_score_split_by_metric.svg"
ax = sns.barplot(
    data=plot_df, x="Dataset", y="value", hue="Metric", palette="tab20",
    order=["Vectra\n Colon", "Vectra\n Pancreas", "MIBI-TOF Breast", "MIBI-TOF Decidua", "CODEX Colon"],
    hue_order=["F1 score_Nimbus", "F1 score_Silver", "Precision_Nimbus", "Precision_Silver", "Recall_Nimbus",
               "Recall_Silver", "Specificity_Nimbus", "Specificity_Silver"], saturation=1
)
plt.xticks(rotation=45)
ax.set_title("Silver standard vs. Nimbus predictions")
# set y axis
ax.set(ylabel='Score')
plt.savefig(os.path.join(out_dir, fig_name), format='svg')
plt.show()